In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc

In [2]:
def extract_emojis(string):
    try:
        return ''.join(c for c in str(string) if c in emoji.UNICODE_EMOJI)
    except TypeError:
        print('wattafak is dat', type(string), string)
        exit()

def __check_conditions(df, mean, std, error=(1,1)):
    target_mean = np.mean(df['num_tracks'])
    target_std = np.std(df['num_tracks'])
    if mean > (target_mean + error[0]) or mean < (target_mean - error[0]):
        print("error m ",mean,target_mean)
        return False
    if std > (target_std + error[1]) or std < (target_std - error[1]):
        print("error s ",std,target_std)
        return False
    return True

def get_random_df_constrained(source_df, num_of_pl, min_v, max_v, mean, std, errors=(1.0, 1.0)):
    """
    iterates until it created a dataframe that satisfies the conditions.
    """
    seed = 0
    while True:
        df = source_df[((source_df['num_tracks']) >= min_v) & ((source_df['num_tracks']) <= max_v)].sample(
            n=num_of_pl, random_state=seed)
        if __check_conditions(df, mean=mean, std=std, error=errors):
            break
        seed+=1
    return df,seed


In [3]:
playlists_train = pd.read_csv("../../../dataset/playlists.csv", delimiter='\t')
interactions = pd.read_csv("../../../dataset/interactions.csv", delimiter='\t')
tracks = pd.read_csv("../../../dataset/tracks.csv", delimiter='\t')


In [4]:
playlists.head()

,name,collaborative,pid,modified_at,num_albums,num_tracks,num_followers,num_tracks.1,num_edits,duration_ms,num_artists,description
0,Throwbacks,False,0,1493424000,47,52,1,52,6,11532414,37,NaN
1,Awesome Playlist,False,1,1506556800,23,39,1,39,5,11656470,21,NaN
2,korean,False,2,1505692800,51,64,1,64,18,14039958,31,NaN
3,mat,False,3,1501027200,107,126,1,126,4,28926058,86,NaN
4,90s,False,4,1401667200,16,17,2,17,7,4335282,16,NaN


In [5]:
interactions.head()

,pid,tid,pos
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4


In [6]:
len(interactions)

66346428

In [7]:
tracks.head()

,tid,arid,alid,track_uri,track_name,duration_ms
0,0,0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),226863
1,1,1,1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,198800
2,2,2,2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,235933
3,3,3,3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,267266
4,4,4,4,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,227600


In [8]:
len(tracks)

2262292

In [9]:
from collections import OrderedDict
cates = {'cat1': (10, 50, 1000,28.6,11.2 ), 'cat2_1': (10, 40, 998,23.8,8.7),
         'cat2_2':(70,80,2,75,4), 'cat3_1': (10, 50,314,29.4,11.4),
         'cat3_2':(51,75,425,62,7.2),'cat3_3':(75,100,261,87,7.1),
         'cat4': (40, 100,1000,63,16.5),'cat5': (40, 100,1000,63.5,17.2),
         'cat6': (40, 100,1000,63.6,16.7),'cat7': (101, 250,1000,150,38.6),
         'cat8': (101, 250,1000,151.7,38.6), 'cat9': (150, 250,1000,189,28),
         'cat_10': (150, 250,1000,187.5,27)}
cates = OrderedDict(sorted(cates.items(), key=lambda t: t[0]))
print ("        name:   (min_value,  max_value, how many to extract, mean,  std)")
cates

        name:   (min_value,  max_value, how many to extract, mean,  std)


OrderedDict([('cat1', (10, 50, 1000, 28.6, 11.2)),
             ('cat2_1', (10, 40, 998, 23.8, 8.7)),
             ('cat2_2', (70, 80, 2, 75, 4)),
             ('cat3_1', (10, 50, 314, 29.4, 11.4)),
             ('cat3_2', (51, 75, 425, 62, 7.2)),
             ('cat3_3', (75, 100, 261, 87, 7.1)),
             ('cat4', (40, 100, 1000, 63, 16.5)),
             ('cat5', (40, 100, 1000, 63.5, 17.2)),
             ('cat6', (40, 100, 1000, 63.6, 16.7)),
             ('cat7', (101, 250, 1000, 150, 38.6)),
             ('cat8', (101, 250, 1000, 151.7, 38.6)),
             ('cat9', (150, 250, 1000, 189, 28)),
             ('cat_10', (150, 250, 1000, 187.5, 27))])

In [10]:
cat_pids = {}
seeds = [0]* len(cates)
count = 0 
for cat, info in cates.items():
    print(cat)
    df, seeds[count] = get_random_df_constrained(playlists_train, min_v=info[0], max_v=info[1], num_of_pl=info[2],
                                                mean=info[3],std=info[4],errors=(1.5, 1.5))
    cat_pids[cat] = list(df.pid)
    playlists_train = playlists_train.drop(df.index)
    count+=1
playlists_train = playlists.reset_index(drop=True)



cat1
cat2_1
cat2_2
error m  75 77.5
error m  75 71.5
cat3_1
cat3_2
cat3_3
cat4
cat5
cat6
cat7
error m  150 152.275
error m  150 154.241
error m  150 152.305
error m  150 153.184
error m  150 152.819
error m  150 153.749
error m  150 152.838
error m  150 152.839
error m  150 153.489
error m  150 152.47
error m  150 151.523
error m  150 153.747
error m  150 151.538
error m  150 154.061
error m  150 152.616
error m  150 151.774
cat8
error m  151.7 154.298
error m  151.7 154.218
error m  151.7 154.784
cat9
cat_10


In [11]:
#save
playlists_train.to_csv("../testset/train_playlists.csv",sep='\t')
gc.collect()
del(playlists_train)

### to generate the same testset you need to use those two variables:
cates, 
seeds

In [12]:
cates

OrderedDict([('cat1', (10, 50, 1000, 28.6, 11.2)),
             ('cat2_1', (10, 40, 998, 23.8, 8.7)),
             ('cat2_2', (70, 80, 2, 75, 4)),
             ('cat3_1', (10, 50, 314, 29.4, 11.4)),
             ('cat3_2', (51, 75, 425, 62, 7.2)),
             ('cat3_3', (75, 100, 261, 87, 7.1)),
             ('cat4', (40, 100, 1000, 63, 16.5)),
             ('cat5', (40, 100, 1000, 63.5, 17.2)),
             ('cat6', (40, 100, 1000, 63.6, 16.7)),
             ('cat7', (101, 250, 1000, 150, 38.6)),
             ('cat8', (101, 250, 1000, 151.7, 38.6)),
             ('cat9', (150, 250, 1000, 189, 28)),
             ('cat_10', (150, 250, 1000, 187.5, 27))])

In [13]:
seeds

[0, 0, 2, 0, 0, 0, 0, 0, 0, 16, 3, 0, 0]

### after having selected the pids of the song to extract, we generate the files

In [14]:
def build_df(pids, title, num_samples):
    pids= cat_pids[cat]
    df = playlists[playlists['pid'].isin(pids)]
    df=df[['name','pid','num_tracks']]
    if not title:
        df['name']=''
    df['num_samples'] = num_samples
    df['num_holdouts'] = df['num_tracks'] - df['num_samples']
    return df

In [15]:
playlists_challange = pd.DataFrame()
test_interactions = pd.DataFrame()
evaluation_interactions = pd.DataFrame()
df_eval_itr= pd.DataFrame()


def build_playlists_df(pids, title, num_samples):
    df = playlists[playlists['pid'].isin(pids)]
    df=df[['name','pid','num_tracks']]
    if not title:
        df['name']=''
    df['num_samples'] = num_samples
    df['num_holdouts'] = df['num_tracks'] - df['num_samples']
    return df


for cat in cates.keys():
    gc.collect()
    pids= cat_pids[cat]
    
    if cat == 'cat1': 
        print(cat,"start", end='')   
        #Predict tracks for a playlist given its title only
        num_samples = 0
        position=True
        title = True

    elif cat == 'cat2_1' or cat=='cat2_2':
        print(cat,"start", end='')     
        #Predict tracks for a playlist given its title and the first track
        num_samples = 1
        position = True
        title = True

    elif cat == 'cat3_1' or cat == 'cat3_2' or cat == 'cat3_3':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its title and the first 5 tracks
        num_samples = 5
        position = True
        title = True

    elif cat == 'cat4':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its first 5 tracks (no title)
        num_samples = 5
        position= False
        title = False

    elif cat == 'cat5':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its title and the first 10 tracks
        num_samples = 10
        position=True
        title = True

    elif cat == 'cat6':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its first ten tracks (no title)
        num_samples = 10
        position=True
        title = False
        
    elif cat == 'cat7':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its title and the first 25 tracks
        num_samples = 25
        title = True
        position= True
        
    elif cat == 'cat8':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its title and 25 random tracks
        num_samples = 25
        title= True
        position = False
        
    elif cat == 'cat9':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its title and the first 100 tracks
        num_samples = 100
        title= True
        position= True
        
    elif cat == 'cat_10':
        print(cat,"start", end='') 
        #Predict tracks for a playlist given its title and 100 random tracks
        num_samples = 100
        title = True
        position = False 
    else:
        raise(Exception,"azz")
    
    print("- build playlist sample", end='')
    df = build_playlists_df(pids, title, num_samples)   
    playlists_challange = pd.concat([playlists_challange,df])
    print("- interactions", end='')
    test_interactions = interactions[interactions['pid'].isin(pids)]
    print("- interactions drop", end='')
    interactions = interactions.drop(test_interactions.index)
    print("- extract songs")
    if position:       
        df_sample = test_interactions[(test_interactions['pos'] >= 0) & (test_interactions['pos'] < num_samples)]
        test_interactions = pd.concat([test_interactions, df_sample])
        test_interactions = test_interactions.drop(df_sample.index) 
        df_eval_itr = pd.concat([df_eval_itr, test_interactions])         
    else:
        for pid in pids:
            df = test_interactions[test_interactions['pid'] == pid]
            df_sample = df.sample(n=num_samples)
            test_interactions = pd.concat([test_interactions, df_sample])
            df = df.drop(df_sample.index)
            df_eval_itr = pd.concat([df_eval_itr, df])
    

cat1 start- build playlist sample- interactions- interactions drop- extract songscat2_1 start- build playlist sample- interactions- interactions drop- extract songscat2_2 start- build playlist sample- interactions- interactions drop- extract songscat3_1 start- build playlist sample- interactions- interactions drop- extract songscat3_2 start- build playlist sample- interactions- interactions drop- extract songscat3_3 start- build playlist sample- interactions- interactions drop- extract songscat4 start- build playlist sample- interactions- interactions drop- extract songscat5 start- build playlist sample- interactions- interactions drop- extract songscat6 start- build playlist sample- interactions- interactions drop- extract songscat7 start- build playlist sample- interactions- interactions drop- extract songscat8 start- build playlist sample- interactions- interactions drop- extract songscat9 start- build playlist sample- interactions- interactions drop- extract songscat_10 start- buil

In [16]:
print('interactions merge')
tids = set(df_eval_itr['tid'])
df = tracks[tracks['tid'].isin(tids)]
df = df[['tid', 'arid']]
df_eval_itr = pd.merge(df_eval_itr, df, on='tid')

print("reset index")
playlists_challange = playlists_challange.reset_index(drop=True)
test_interactions = test_interactions.reset_index(drop=True)
df_eval_itr = df_eval_itr.reset_index(drop=True)
interactions = interactions.reset_index(drop=True) 


interactions merge
reset index


# print files


In [17]:
print(playlists_challange.head())
playlists_challange.to_csv("challange_playlists.csv", sep='\t')
del(playlists_challange)

              name     pid  num_tracks  num_samples  num_holdouts
0             Funk   10000          16            0            16
1              Rap  101077          15            0            15
2  One Hit Wonders  101650          34            0            34
3            Metal  101934          48            0            48
4       The Voice   103915          10            0            10


In [21]:
print(df_eval_itr.head())
df_eval_itr.to_csv("evaluation_interactions.csv", sep='\t')
del(df_eval_itr)

      pid   tid  pos  arid
0   10000  9129    0   912
1  608072  9129   19   912
2  685211  9129   31   912
3  887395  9129   13   912
4   33973  9129   15   912


In [ ]:
print(interactions.head())
df_.to_csv("train_interactions.csv", sep='\t')
del(interactions)

   pid  tid  pos
0    0    0    0
1    0    1    1
2    0    2    2
3    0    3    3
4    0    4    4


In [2]:
print(test_interactions.head())
test_interactions.to_csv("test_interactions.csv", sep='\t')
del(test_interactions)


NameError: name 'test_interactions' is not defined